In [ ]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

from tensorboard.plugins.hparams import api as hp
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import backend


In [ ]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from src.extract_to_df import extract_to_df
from src.visualizations import plot_graph
from src.metric_notes import metric_note

In [ ]:
#prevent a bug that some images are truncated
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
test = Path('Data/test')
train = Path('Data/train')
valid = Path('Data/valid')


In [ ]:
train_df = extract_to_df(train, 'Train')
test_df = extract_to_df(test, 'Test')
val_df = extract_to_df(valid,'Valid')

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

test_generator = ImageDataGenerator(
    rescale=1./255
)


In [ ]:
pixel = 64
size = f'{pixel}x{pixel}'

In [ ]:
train_images = train_generator.flow_from_dataframe(dataframe=train_df,
                                                   x_col='Path',
                                                   y_col='Label',
                                                   target_size=(pixel, pixel),
                                                   class_mode='binary',
                                                   color_mode='rgb',
                                                   shuffle=True,
                                                   seed=42,
                                                   batch_size=128)

valid_images = train_generator.flow_from_dataframe(dataframe=val_df,
                                                   x_col='Path',
                                                   y_col='Label',
                                                   target_size=(pixel, pixel),
                                                   class_mode='binary',
                                                   color_mode='rgb',
                                                   shuffle=True,
                                                   seed=42,
                                                   batch_size=64)

test_images = test_generator.flow_from_dataframe(dataframe=test_df,
                                                 x_col='Path',
                                                 y_col='Label',
                                                 target_size=(pixel, pixel),
                                                 class_mode='binary',
                                                 color_mode='rgb',
                                                 shuffle=True,
                                                 seed=42,
                                                 batch_size=128)

In [ ]:
metrics = metric_note(train_images, test_images, valid_images)

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_LEARNING = hp.HParam('learning', hp.Discrete([0.001, 0.005,0.0001]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_LEARNING],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
def create_model(hparams):
    cnn4 = Sequential([
        Conv2D(hparams[HP_NUM_UNITS], kernel_size =(3,3) , activation = 'relu', input_shape=(pixel, pixel, 3)),
        MaxPooling2D(pool_size=2),
        BatchNormalization(),
        Dropout(hparams[HP_DROPOUT]),
        Conv2D(hparams[HP_NUM_UNITS]*2, kernel_size =(3,3) , activation = 'relu', input_shape=(pixel, pixel, 3)),
        MaxPooling2D(pool_size=2),
        BatchNormalization(),
        Dropout(hparams[HP_DROPOUT]),
        Conv2D(hparams[HP_NUM_UNITS]*3, kernel_size =(3,3) , activation = 'relu', input_shape=(pixel, pixel, 3)),
        MaxPooling2D(pool_size=2),
        BatchNormalization(),
        Dropout(hparams[HP_DROPOUT]),
        Conv2D(hparams[HP_NUM_UNITS]*4, kernel_size =(3,3) , activation = 'relu', input_shape=(pixel, pixel, 3)),
        MaxPooling2D(pool_size=2),
        BatchNormalization(),
        Dropout(hparams[HP_DROPOUT]*4),
        Flatten(),
        Dense(hparams[HP_NUM_UNITS], activation = 'relu'),
        BatchNormalization(),
        Dropout(hparams[HP_DROPOUT]),
        Dense(1, activation='sigmoid')
    ])
    cnn4.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    cnn4.fit(train_images,
            batch_size=64,
            epochs=10)
    _, accuracy = cnn4.evaluate(test_images)
    return accuracy


In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  
    accuracy = create_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for learning in HP_LEARNING.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_LEARNING: learning,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1